In [6]:
# Importing dependecies:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

import gensim.models as gs
from gensim.models.phrases import Phrases, Phraser

from numpy import array
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


## Vectorizing Columns:

In [8]:
df['ingredients_vector'] = gs.Word2Vec(df['ingredients_vector'], min_count=1, size= 50, workers=3, window=3, sg=1)
df['instructions_vector'] = gs.Word2Vec(df['instructions_vector'], min_count=1, size= 50, workers=3, window=3, sg=1)

In [10]:
print(df['ingredients_vector'])
data = df['ingredients_vector'].most_similar('chicken')
print(data)

p3pKOD6jIHEcjf20CCXohP8uqkG5dGi    Word2Vec(vocab=56508, size=50, alpha=0.025)
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO    Word2Vec(vocab=56508, size=50, alpha=0.025)
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK    Word2Vec(vocab=56508, size=50, alpha=0.025)
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK    Word2Vec(vocab=56508, size=50, alpha=0.025)
kRBQSWtqYWqtkb34FGeenBSbC32gIdO    Word2Vec(vocab=56508, size=50, alpha=0.025)
                                                      ...                     
4bfMWxlbKhx/McJq/89k0SBdw.VvAzW    Word2Vec(vocab=56508, size=50, alpha=0.025)
T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m    Word2Vec(vocab=56508, size=50, alpha=0.025)
f/coffo2TMs2J2gq5nTOUIqH2TRAkui    Word2Vec(vocab=56508, size=50, alpha=0.025)
q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS    Word2Vec(vocab=56508, size=50, alpha=0.025)
7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm    Word2Vec(vocab=56508, size=50, alpha=0.025)
Name: ingredients_vector, Length: 59612, dtype: object


AttributeError: 'Series' object has no attribute 'most_similar'

In [ ]:
X = df_tokenized.drop('title', axis=1)
y = df_tokenized['title'].values
print(X.shape, y.shape)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, y_train.shape

In [ ]:
tokenizer = Tokenizer(num_words=1000)
# x_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
# x_test = tokenizer.sequences_to_matrix(X_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3)